# 준비물

In [73]:
!pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.4.5-cp37-none-any.whl size=37983 sha256=13b5de1ff5251e19be9eeacb8c956471a8640fa2f4d66c4a17982d9a630eb34f
  Stored in directory: /root/.cache/pip/wheels/fb/24/33/51b0faba1170f6519eae3c51b6adbd9939296c73f54230b069
Successfully built googlemaps


In [1]:
!pip install tensorflow-addons

     |████████████████████████████████| 706kB 26.7MB/s 


In [2]:
!pip install python-Levenshtein
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, SpatialDropout1D, LayerNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import warnings
import ast
import numpy as np
import gensim
from gensim.models import FastText
from gensim import models
from gensim.test.utils import datapath
warnings.filterwarnings('ignore')

     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149818 sha256=2ccbf61856f1412f30288fc23cd1a93b162c00aa3d175b8512801ef79c986c1b
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import os
os.chdir('gdrive/Shareddrives/DEEPTEAMZZANG/RECOMMENDATION')

# 유저벡터 만들기

## (0) 좌표를 붙여주자!

In [52]:
pyo = pd.read_csv('plc_with_xy.csv')

In [60]:
pyo

,name,x,y
0,우도(해양도립공원),126.951017,33.519472
1,성산일출봉(UNESCO 세계자연유산),126.936801,33.462234
2,사려니숲길,126.642509,33.407706
3,카멜리아힐,126.368365,33.290140
4,협재해수욕장,126.239432,33.393748
...,...,...,...
344,제주난타,126.547910,33.445827
345,제주항일기념관,126.640726,33.541027
346,평화통일 불사리탑,126.636878,33.531510
347,관음사 템플스테이,126.558751,33.421720


## (1) tag별 평균벡터

In [19]:
tag_vec = pd.read_csv('tag_vec.csv')

In [20]:
tag_vec

,plc,mean_vec
0,4.3,[9.81627612e-02 1.37100981e-01 1.57903474e+00 ...
1,가을,[0.65210563 0.26734542 0.25313658 0.18481011 0...
2,감귤,[0.00000000e+00 5.33083856e-01 0.00000000e+00 ...
3,건물,[5.42201474e-02 2.74108529e-01 9.41252708e-02 ...
4,걷기,[0.37753223 0.25135711 0.21834417 0.60891551 0...
...,...,...
121,혼자,[1.15090094 0.10228937 0.19172733 0.13588028 0...
122,휴식/힐링,[0.55283896 0.3533469 0.23095974 0.15937136 0...
123,휴양림,[1.73887394e-02 1.92017345e-01 7.12568629e-02 ...
124,흐림,[0.20500935 0.19900645 0.55412866 0.23510308 0...


In [24]:
for i in range(tag_vec.shape[0]):
  tag_vec['mean_vec'][i] = [i.translate({ord('['):"", ord(']'):""}) for i in tag_vec['mean_vec'][i].split()]
tag_vec['mean_vec'] = tag_vec['mean_vec'].apply(lambda x : [float(x[i]) for i in range(300)])
tag_vec['mean_vec'] = tag_vec['mean_vec'].apply(lambda x : np.asarray(x))
type(tag_vec['mean_vec'][0])

## (2) index로 tag이름 주기

In [39]:
tag_vec.set_index(keys = tag_vec.plc, inplace=True)
tag_vec.drop(columns='plc', inplace=True)

## (3) User vector 만들기

In [45]:
type(np.asarray(tag_vec.loc['희생'][0]))

numpy.ndarray

In [46]:
def calc_user_vector(df, key_words_list):
  user_vector = np.empty((1,300))
  tmp = pd.DataFrame(columns = ['vectors'])
  tmp['vectors'] = [ np.asarray(df.loc[words][0]) for words in key_words_list ]
  user_vector = tmp['vectors'].mean()
  return user_vector

In [61]:
user_vec = calc_user_vector(df=tag_vec, key_words_list = ['희생','혼자', '휴식/힐링'])

## (4) 관광지 벡터 불러오기

In [63]:
plc_vec_df = pd.read_csv('mean_vec_BiLSTM.csv')
plc_vec_df['mean_vec'] = plc_vec_df['mean_vec'].apply(lambda x: np.asarray(ast.literal_eval(x)))

In [66]:
plc_vec_df['x'] = pyo['x']
plc_vec_df['y'] = pyo['y']
del pyo
plc_vec_df.head()

,plc,mean_vec,x,y
0,1100고지(휴게소),"[0.0, 0.464537114, 0.0965782329, 0.956899107, ...",126.951017,33.519472
1,1100고지습지,"[0.50620675, 0.0, 1.0605601, 0.77088135, 0.0, ...",126.936801,33.462234
2,가파도,"[0.0481949858, 0.581190646, 1.50552094, 0.4385...",126.642509,33.407706
3,감귤박물관,"[0.384193838, 0.140640646, 0.0, 0.229497015, 0...",126.368365,33.290140
4,거문오름(UNESCO 세계자연유산),"[3.12251568, 0.295921564, 0.18568857, 0.181027...",126.239432,33.393748


# 유저벡터-관광지 유사도 행렬




In [67]:
def cos(user_vec, plc_vec):
  return np.dot(user_vec, plc_vec) / (np.linalg.norm(user_vec) * (np.linalg.norm(plc_vec)))

In [68]:
def get_sim (user_vec, plc_vec_df, vec_col_name, plc_col_name):
  '''
  user_vec : user_vec
  plc_vec_df : plc_vec가 들어 있는 dataframe
  column_name : plc_vec가 들어 있는 column의 이름
  '''
  dist = [cos(user_vec, plc_vec_df[vec_col_name][i]) for i in range(0,len(plc_vec_df))]
  sim_df = pd.DataFrame(columns = ['plc', 'sim'])
  for i in range(0,len(plc_vec_df)):
   sim_df = sim_df.append({'plc' : plc_vec_df[plc_col_name][i],
                         'sim' : dist[i],
                         'x' : plc_vec_df['x'][i],
                         'y' : plc_vec_df['y'][i]}, ignore_index=True)
  sim_df.sort_values(by='sim', ascending=False, inplace=True)
  return sim_df.reset_index(drop=True)

In [69]:
trial_1 = get_sim(user_vec, plc_vec_df, 'mean_vec', 'plc')

In [71]:
trial_1

,plc,sim,x,y
0,섭지코지,0.689866,126.299661,33.304207
1,산방산,0.652927,126.558424,33.239201
2,비양도,0.652466,126.310111,33.462749
3,산굼부리,0.646883,126.489686,33.452550
4,송악산,0.641998,126.265692,33.339693
...,...,...,...,...
344,탐나라공화국(제주남이섬),0.404070,126.349729,33.404352
345,교래자연휴양림,0.403590,126.736551,33.557208
346,중문골프장,0.397501,126.882705,33.526853
347,전농로 서사라문화거리,0.358379,126.393124,33.409841


# 거리기반 추천

In [74]:
from cal_dist import *

In [75]:
rec = trial_1[:10]
location = [rec.iloc[0]['plc']]
meter = []
ms = []

In [76]:
start = str(rec.iloc[0]['x']) + ',' + str(rec.iloc[0]['y'])
loop = rec.shape[0]-1
for j in range(loop):
    dic = {}
    rec = rec[rec['plc'] != location[-1]]
    for i in range(rec.shape[0]):
        end = str(rec.iloc[i]['x']) + ',' + str(rec.iloc[i]['y'])
        dic[rec.iloc[i]['plc']] = with_driving(start, end,'a21z0up68q', 'wfi13MHklam5QjsbAMFoye5vBh3Gp2qZgbvSwSvb')
    res = sorted(dic.items(), key=lambda item: item[1][0][0])[0]
    location.append(res[0])
    meter.append(res[1][0][0])
    ms.append(res[1][1][0])
    tmp = rec[rec['plc']==res[0]]
    start = str(tmp.iloc[0]['x'])+','+str(tmp.iloc[0]['y'])

In [ ]:
d